# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [11]:
%load_ext autoreload
%autoreload 2
from helpers.irp_integration import IRPClient
import json, os
from pathlib import Path

irp_client = IRPClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Configurations

In [28]:
edm_name = "integration_demo" # ensure unique within IRP
portfolio_name = "demo_portfolio" # ensure unique within the edm
accounts_file_name = "Modeling_202503_Moodys_CBHU_Account.csv"
locations_file_name = "Modeling_202503_Moodys_CBHU_Location.csv"
analysis_job_name = "basic_job"
analysis_profile_name = "North Atlantic Hurricane 18.0 EP Distributed Wind Only (Tutorial)"
output_profile_name = "Default"
event_rate_scheme_name = "RMS 2023 Stochastic Event Rates"

### Create EDM

In [4]:
print(json.dumps(irp_client.edm.create_edm(edm_name, "databridge-1"), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786608
Workflow status: PENDING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786608
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786608
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786608
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786608
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786608
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786608
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/ris

### Upgrade EDM Version

In [6]:
print(json.dumps(irp_client.edm.upgrade_edm_version(edm_name), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786625
Workflow status: PENDING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786625
Workflow status: RUNNING; Percent complete: 100
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786625
Workflow status: FINISHED; Percent complete: 100
{
  "id": 22786625,
  "userName": "Partner - PremiumIQ key",
  "status": "FINISHED",
  "submitTime": "2025-10-16T01:12:22.247Z",
  "startTime": "2025-10-16T01:12:23Z",
  "endTime": "2025-10-16T01:12:34Z",
  "name": "integration_demo",
  "type": "EDM_DATA_UPGRADE",
  "progress": 100,
  "priority": "medium",
  "entitlement": "RI-RISKMODELER",
  "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
  "jobs": [
    {
      "id": "c8d81e10-b5d4-4724-90e3-c40a4c68fdf6",
      "taskId": 1,
      "workflowId": 22786625,
      "status": "Succeeded",
      "submitTime": "2

### Create Portfolio

In [7]:
portfolio_response = irp_client.portfolio.create_portfolio(edm_name, portfolio_name)
portfolio_id = portfolio_response['id'].split('/')[-1]
print(json.dumps(portfolio_response, indent=2))

{
  "id": "1"
}


### MRI Import

In [14]:
notebook_dir = Path(os.getcwd())
working_files_dir = notebook_dir / "workspace" / "workflows" / "_Tools" / "files" / "working_files"

bucket_response = irp_client.mri_import.create_aws_bucket()
bucket_url = bucket_response.headers['location']
bucket_id = bucket_url.split('/')[-1]

accounts_file_path = os.path.join(working_files_dir, accounts_file_name)
accounts_credentials = irp_client.mri_import.get_file_credentials(bucket_url, accounts_file_name, 100, "account")
irp_client.mri_import.upload_file_to_s3(accounts_credentials, accounts_file_path)

locations_file_path = os.path.join(working_files_dir, locations_file_name)
locations_credentials = irp_client.mri_import.get_file_credentials(bucket_url, locations_file_name, 100, "location")
irp_client.mri_import.upload_file_to_s3(locations_credentials, locations_file_path)

mapping_file_name = "mapping.json"
mapping_file_path = os.path.join(working_files_dir, mapping_file_name)
mapping_file_id = irp_client.mri_import.upload_mapping_file(mapping_file_path, bucket_id).json()

import_response = irp_client.mri_import.execute_mri_import(
    edm_name,
    int(portfolio_id),
    int(bucket_id),
    accounts_credentials['file_id'],
    locations_credentials['file_id'],
    mapping_file_id
)
print(json.dumps(import_response, indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786638
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786638
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786638
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786638
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786638
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786638
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22786638
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodele

### Create Treaty

In [31]:
treaty_name = "test_treaty"
treaty_type_name = "Working Excess"
currency_name = "US Dollar"
attachment_basis = "Losses Occurring"
attachment_level = "Policy"

cedant_data = irp_client.edm.get_cedants_by_edm(edm_name)["searchItems"]
lob_data = irp_client.edm.get_lobs_by_edm(edm_name)["searchItems"]

treaty_type_data = irp_client.treaty.get_treaty_types_by_edm(edm_name)["entityItems"]["values"]
treaty_type_match = next(
    (item for item in treaty_type_data if item["name"] == treaty_type_name),
    None  # fallback if no match found
)

attachment_basis_data = irp_client.treaty.get_treaty_attachment_bases_by_edm(edm_name)["entityItems"]["values"]
attachment_basis_match = next(
    (item for item in attachment_basis_data if item["name"] == attachment_basis),
    None  # fallback if no match found
)

attachment_level_data = irp_client.treaty.get_treaty_attachment_levels_by_edm(edm_name)["entityItems"]["values"]
attachment_level_match = next(
    (item for item in attachment_level_data if item["name"] == attachment_level),
    None  # fallback if no match found
)

currency_data = irp_client.reference_data.get_currencies()["entityItems"]["values"]
currency_match = next(
    (item for item in currency_data if item["name"] == currency_name),
    None  # fallback if no match found
)

if len(cedant_data) > 0 and len(lob_data) > 0 and treaty_type_match and attachment_basis_match and attachment_level_match and currency_match:
    data = {
        "treatyNumber": treaty_name,
        "treatyName": treaty_name,
        "treatyType": treaty_type_match,
        "riskLimit": 3000000,
        "occurLimit": 9000000,
        "attachPt": 2000000,
        "cedant": cedant_data[0],
        "effectDate": "2025-10-15T17:49:10.637Z",
        "expireDate": "2026-10-15T17:49:10.637Z",
        "currency": {'code': currency_match['code'], 'name': currency_match['name']},
        "attachBasis": attachment_basis_match,
        "attachLevel": attachment_level_match,
        "pcntCovered": 100,
        "pcntPlaced": 95,
        "pcntRiShare": 100,
        "pcntRetent": 100,
        "premium": 0,
        "numOfReinst": 99,
        "reinstCharge": 0,
        "aggregateLimit": 0,
        "aggregateDeductible": 0,
        "priority": 1,
        "retentAmt": "",
        "isValid": True,
        "userId1": "",
        "userId2": "",
        "maolAmount": "",
        "lobs": lob_data,
        "tagIds": []
    }

    # print(json.dumps(data, indent=2))

    treaty_response = irp_client.treaty.create_treaty(edm_name, data)
    lob_ids = []
    for lob in lob_data:
        lob_ids.append(lob['id'])
    print(json.dumps(irp_client.treaty.assign_lobs(edm_name, treaty_response['id'], lob_ids)))
    

[{"context": {"headers": {}, "entityAnnotations": [], "entityStream": {"closed": false, "committed": false}, "length": -1, "acceptableMediaTypes": [{"type": "*", "subtype": "*", "parameters": {}, "quality": 1000, "wildcardType": true, "wildcardSubtype": true}], "acceptableLanguages": ["*"], "allowedMethods": [], "links": [], "stringHeaders": {}, "committed": false, "requestCookies": {}, "lengthLong": -1, "responseCookies": {}}, "status": 201, "length": -1, "statusInfo": "CREATED", "metadata": {}, "allowedMethods": [], "cookies": {}, "links": [], "stringHeaders": {}, "headers": {}}]


### Geocode

In [ ]:
print(json.dumps(
    irp_client.portfolio.geohaz_portfolio(edm_name, 
                                          portfolio_id, 
                                          geocode=True, 
                                          hazard_eq=True, 
                                          hazard_ws=True), 
    indent=2
))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/risk

### Analysis

In [19]:
analysis_response = irp_client.analysis.execute_analysis(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name
)
print(json.dumps(analysis_response, indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodele